In [9]:
!nvidia-smi

Mon Apr 29 22:38:09 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.171.04             Driver Version: 535.171.04   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce GTX 1650        Off | 00000000:01:00.0 Off |                  N/A |
| N/A   53C    P8               4W /  50W |    584MiB /  4096MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
!pip install supervision==0.2.0

from IPython import display
display.clear_output()

import supervision as sv
print("supervision", sv.__version__)

supervision 0.2.0


In [3]:
from ultralytics import YOLO

model = YOLO('yolov8s.pt')

In [4]:
import os
HOME = os.getcwd()
print(HOME)

/home/sukuna/Documents/PROJECTS/Dashcamdetector


In [6]:
%cd {HOME}
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1M3UuH3QNDWGiH0NmGgHtIgXXGDo_nigm' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1M3UuH3QNDWGiH0NmGgHtIgXXGDo_nigm" -O mall.mp4 && rm -rf /tmp/cookies.txt

/home/sukuna/miniconda3/envs/fish_tank/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


/home/sukuna/Documents/PROJECTS/Dashcamdetector
--2024-04-29 21:03:37--  https://docs.google.com/uc?export=download&confirm=&id=1M3UuH3QNDWGiH0NmGgHtIgXXGDo_nigm
Resolving docs.google.com (docs.google.com)... 142.250.195.174, 2404:6800:4007:826::200e
Connecting to docs.google.com (docs.google.com)|142.250.195.174|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1M3UuH3QNDWGiH0NmGgHtIgXXGDo_nigm&export=download [following]
--2024-04-29 21:03:40--  https://drive.usercontent.google.com/download?id=1M3UuH3QNDWGiH0NmGgHtIgXXGDo_nigm&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 142.250.195.193, 2404:6800:4007:827::2001
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|142.250.195.193|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 103282086 (98M) [video/mp4]
Saving to: ‘mall.mp4’

mall.mp4            100%[============

In [5]:
MALL_VIDEO_PATH = f"{HOME}/mall.mp4"

In [12]:
import supervision as sv
import cv2 as cv
import matplotlib.pyplot as plt

# extract video frame
generator = sv.get_video_frames_generator(MALL_VIDEO_PATH)
iterator = iter(generator)
frame = next(iterator)

# detect
results = model(frame, imgsz=1280)[0]
detections = sv.Detections.from_ultralytics(results)

# annotate
box_annotator = sv.BoxAnnotator(thickness=4, text_thickness=4, text_scale=2)
frame = box_annotator.annotate(scene=frame, detections=detections)

%matplotlib inline  
# sv.show_frame_in_notebook(frame, (16, 16))

x = cv.imread(frame)
plt.imshow(x)


0: 736x1280 1 person, 4 bottles, 1 chair, 1 tv, 2 refrigerators, 39.5ms
Speed: 11.8ms preprocess, 39.5ms inference, 1.1ms postprocess per image at shape (1, 3, 736, 1280)


SupervisionWarnings: annotate is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead


TypeError: Can't convert object to 'str' for 'filename'

In [6]:
import supervision as sv

import numpy as np
np.bool = np.bool_



# extract video frame
generator = sv.get_video_frames_generator(MALL_VIDEO_PATH)
iterator = iter(generator)
frame = next(iterator)

# detect
results = model(frame, imgsz=1280)[0]
detections = sv.Detections.from_yolov8(results)
detections = detections[detections.class_id == 0]

# annotate
box_annotator = sv.BoxAnnotator(thickness=4, text_thickness=4, text_scale=2)
labels = [f"{model.names[class_id]} {confidence:0.2f}" for _, confidence, class_id, _ in detections]
frame = box_annotator.annotate(scene=frame, detections=detections, labels=labels)

%matplotlib inline  
sv.show_frame_in_notebook(frame, (16, 16))


0: 736x1280 1 person, 4 bottles, 1 chair, 1 tv, 2 refrigerators, 46.7ms
Speed: 8.7ms preprocess, 46.7ms inference, 1520.3ms postprocess per image at shape (1, 3, 736, 1280)


AttributeError: type object 'Detections' has no attribute 'from_yolov8'

In [ ]:
# initiate polygon zone
polygon = np.array([
    [1900, 1250],
    [2350, 1250],
    [3500, 2160],
    [1250, 2160]
])
video_info = sv.VideoInfo.from_video_path("dash.mp4")
zone = sv.PolygonZone(polygon=polygon, frame_resolution_wh=video_info.resolution_wh)

# initiate annotators
box_annotator = sv.BoxAnnotator(thickness=4, text_thickness=4, text_scale=2)
zone_annotator = sv.PolygonZoneAnnotator(zone=zone, color=sv.Color.white(), thickness=6, text_thickness=6, text_scale=4)

# extract video frame
generator = sv.get_video_frames_generator(MALL_VIDEO_PATH)
iterator = iter(generator)
frame = next(iterator)

# detect
results = model(frame, imgsz=1280)[0]
detections = sv.Detections.from_yolov8(results)
detections = detections[detections.class_id == 0]
zone.trigger(detections=detections)

# annotate
box_annotator = sv.BoxAnnotator(thickness=4, text_thickness=4, text_scale=2)
labels = [f"{model.names[class_id]} {confidence:0.2f}" for _, confidence, class_id, _ in detections]
frame = box_annotator.annotate(scene=frame, detections=detections, labels=labels)
frame = zone_annotator.annotate(scene=frame)

%matplotlib inline  
sv.show_frame_in_notebook(frame, (16, 16))

In [14]:
# generator = sv.get_video_frames_generator(MALL_VIDEO_PATH)
# iterator = iter(generator)
# frame = next(iterator)
print(frame)

[[[133 143 147]
  [133 143 147]
  [133 143 147]
  ...
  [ 18 198 251]
  [ 21 196 250]
  [ 21 196 250]]

 [[133 143 147]
  [133 143 147]
  [133 143 147]
  ...
  [ 18 198 251]
  [ 21 196 250]
  [ 21 196 250]]

 [[133 143 147]
  [133 143 147]
  [133 143 147]
  ...
  [ 14 198 251]
  [ 17 197 250]
  [ 17 197 250]]

 ...

 [[ 35  28  34]
  [ 35  28  34]
  [ 35  28  34]
  ...
  [ 44  56  43]
  [ 44  56  43]
  [ 44  56  43]]

 [[ 42  33  44]
  [ 42  33  44]
  [ 42  33  44]
  ...
  [ 44  56  43]
  [ 44  56  43]
  [ 44  56  43]]

 [[ 45  36  47]
  [ 45  36  47]
  [ 45  36  47]
  ...
  [ 44  56  43]
  [ 44  56  43]
  [ 44  56  43]]]
